In [123]:
import nltk
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer

import pandas as pd
import pickle
import numpy as np
import os
import matplotlib.pyplot as plt
import gensim
from IPython.display import display, clear_output


%matplotlib.inline

[nltk_data] Downloading package wordnet to /home/skp454/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
UsageError: Line magic function `%matplotlib.inline` not found.


In [4]:
old = ['hindiMedium.csv', 'bareillyKiBarfi.csv', 'simran.csv', 'newton.csv', 'sanju.csv']

In [5]:
## Find list of csv
csvs = []
for i in os.listdir('/scratch/skp454/AST/MainDir/data/'):
    try:
        if i.split('.')[1] == 'csv':
            csvs.append(i)
    except:
        pass
print(csvs)

['bajrangiBhaijaan.csv', 'gabbarIsBack.csv', 'brothers.csv', 'hindiMedium.csv', 'dilwale.csv', 'khoobsurat.csv', 'bandBaajaBaraat.csv', 'dilDhadakneDo.csv', 'bareillyKiBarfi.csv', 'madrasCafe.csv', 'simran.csv', 'newton.csv', 'aurangzeb.csv', 'sanju.csv', 'bhoothnathReturns.csv']


In [6]:
## Extract text and make dictionary
words = {}
sentences =[]
for i in csvs:
    file = pd.read_csv( '/scratch/skp454/AST/MainDir/data/'+i , header = None)
    file[4] = i.split('.')[0] + file[4]
    for j in file.index:
        sentences.append(file.loc[j,3])
        for word in file.loc[j,3].split(' '):
            try:
                words[word] += 1
            except KeyError:
                words[word] = 1

In [106]:
def lemmatize(word):
    avoid = ['has', 'was', 'as', 'us', 'pass', 'does', 'boss', 'less', 'discuss']
    if word not in avoid:
        if WordNetLemmatizer().lemmatize(word, 'n') is not word:
            word = WordNetLemmatizer().lemmatize(word,'n')
    word = WordNetLemmatizer().lemmatize(word,'v')
    return word
    

In [108]:
# clean up using NLTK to prepare words for word2vec embedding
new_words = {}

for word in words.keys():
    count = words[word]
    word = lemmatize(word)
    try:
        new_words[word] += count
    except KeyError:
        new_words[word] = count


In [110]:
len(list(new_words.keys()))/len(list(words.keys())) * 100

76.9672131147541

In [16]:
## Import word 2 vec embedding
# embed = gensim.models.KeyedVectors.load_word2vec_format('/scratch/skp454/bigDataML/GoogleNews-vectors-negative300.bin', binary=True)  

In [43]:
# w2v_words ={}
# a = 0
# for i in new_words:
#     a+=1
#     if i in embed.index2word:
#         w2v_words[i] = new_words[i]
#     if a % 50 == 0:
#         clear_output()
#         print(a*100/9670)
# print(len(list(w2v_words.keys()))/len(list(words.keys())))
# print(len(list(new_words.keys())))
# print(sum(list(w2v_words.values()))/sum(list(words.values())))

77.55946225439503


In [116]:
# count loss if less occuring words are removed
thres = 10

values = list(new_words.values())
values.sort()
df = pd.DataFrame(pd.Series(values).value_counts())
df[1] = df[0] * df.index
df[2] = (np.cumsum(df[1])/sum(list(words.values())))
df[3] = (np.cumsum(df[0])/sum(df[0]))
print('Total words lost: ', str(df[df.index==thres][2][thres]*sum(list(words.values()))), ' % Lost: ',str(df[df.index==thres][2][thres]*100))
print('Total Vocab lost: ', int(df[df.index==thres][3][thres]*len(list(words.keys()))), ' % Lost: ',df[df.index==thres][3][thres]*100)

final_words = {}
for word in new_words:
    if new_words[word] > 10:
        final_words[word] = new_words[word]
print('Length of new dict: ', len(list(final_words.keys())))
print('Total words: ', sum(list(final_words.values())))

Total words lost:  15515.0  % Lost:  9.44947590886
Total Vocab lost:  8044  % Lost:  82.428115016
Length of new dict:  1320
Total words:  148674


In [128]:
new_sentences =[]
for i in csvs:
    file = pd.read_csv( '/scratch/skp454/AST/MainDir/data/'+i , header = None)
    file[4] = i.split('.')[0] + file[4]
    for j in file.index:
        include = False
        sent  = []
        for word in file.loc[j,3].split(' '):
            word = lemmatize(word)
            if word in list(final_words.keys()):
                include = True
            else:
                word = '_'
            sent.append(word)
        if include == True:
            new_sentences.append(sent)
print('Total Sentences lost:', len(sentences) - len(new_sentences),\
      '% Sentences lost: ', (len(sentences) - len(new_sentences))/len(sentences))

Total Sentences lost: 519 % Sentences lost:  0.017918795746443863


In [124]:
# Save strength of words in dictionary
outfile = open('/scratch/skp454/AST/MainDir/data/'+'dictionary_str.pkl','wb')
pickle.dump(final_words ,outfile)
outfile.close()

In [129]:
# Create dictionary mapping
dictionary = {'<s>':0,'</s>':1,'_':2}
a = 3
for word in list(final_words.keys()):
    dictionary[word] = a
    a +=1
del dictionary['']

In [130]:
# Save dictionary
outfile = open('/scratch/skp454/AST/MainDir/data/'+'dictionary.pkl','wb')
pickle.dump(dictionary ,outfile)
outfile.close()